<a href="https://colab.research.google.com/github/GitYCC/ai_playground/blob/master/kick_off_gnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 跨入Graph Neural Network的第一檻

最近深度學習領域吹起一波Graph Neural Network (GNN)的風潮，大家到底為何而瘋呢？

在概念上Graph是更一般化的命題，不管是圖像或是序列都可以看作是Graph的一種特例，所以GNN成為了取代CNN或RNN的明日之星，也因此GNN有範圍更大的使用場景，將可以擴大使用到行銷廣告、社交網路、生物、化學、物理、...等等，甚至很可能可以解決因果推理難題。綜上所述，GNN成為了機器學習領域新的必爭之地

## 設置 torch-geometric


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


目前cuda版本是10.1，所以根據版本如下安裝（應自行調配）

In [3]:
%env CUDA=cu101
!pip install -q torch==1.5.0+${CUDA} torchvision==0.6.0+${CUDA} -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q torch-scatter==latest+${CUDA} -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install -q torch-sparse==latest+${CUDA} -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install -q torch-cluster==latest+${CUDA} -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install -q torch-spline-conv==latest+${CUDA} -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install -q torch-geometric

env: CUDA=cu101
     |████████████████████████████████| 12.3MB 236kB/s 
     |████████████████████████████████| 24.4MB 121kB/s 
     |████████████████████████████████| 18.2MB 175kB/s 
     |████████████████████████████████| 6.3MB 6.9MB/s 
     |████████████████████████████████| 153kB 4.8MB/s 
     |████████████████████████████████| 235kB 8.1MB/s 
     |████████████████████████████████| 2.1MB 10.9MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils
from torch_geometric.data import DataLoader
import torch_geometric.transforms as T

## 什麼是Graph?



## 載入Graph Dataset

我們首先使用[Cora dataset](https://relational.fit.cvut.cz/dataset/CORA)來作示範，Cora dataset包含2708篇的科學論文，這些論文可以分為7類，這些論文間會彼此引用，資料集當中總共含有5278個引用連結，另外資料中會用一條只含有0/1的Word Vector來當作每篇論文的feature，Word Vector的長度總共有1433個，代表著是否存在1433個不重複的字詞，假設論文中有出現這個字詞那個位置就標示1，否則標示為0。

我們用Graph的概念來理解這個dataset，每一篇論文可以當作Node，而每個Node都含有Word Vector當作Node Feature，而將這些Node連接起來的Edge就是引用連結。給定這樣的Graph，我們想辦法去預測這篇論文（Node）是屬於哪一類。

In [5]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [6]:
len(dataset)  # Graph的數量

1

In [7]:
dataset.name

'Cora'

In [8]:
dataset.num_classes  # 類別數量

7

In [9]:
dataset.num_node_features  # Node Feature的數量

1433

In [0]:
graph = dataset[0]

In [11]:
graph.is_undirected()  # 判斷Graph是不是無向

True

In [12]:
graph

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

以上有很多Graph的資訊，我們逐一來看：

- `edge_index` 描述Edge的連接狀況

In [13]:
graph.edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])

從上面的例子，`(0, 633)` 代表是從 Node 0 朝 Node 633 的 Edge，`(0, 1862)` 代表是從 Node 0 朝 Node 1862 的 Edge，以此類推。因為我們這個 Graph 是無向的，所以還有另外一個方向的 Edge `(633, 0)` 和 `(1862, 0)` ，因此雖然 Edge 數量為 5278，但是 `graph.edge_index.shape[1]` 則是 10556。  

- `test_mask`, `train_mask`, `val_mask` 則是用來切分 Testing Set、Train Set 和 Validation Set的遮罩

In [23]:
graph.test_mask

tensor([False, False, False,  ...,  True,  True,  True])

- `x` 表示 Node Feature

In [24]:
graph.x

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

總共有2708個Node，每個Node都有1433維的Feature。

- `y`: Node的類別

In [25]:
graph.y

tensor([3, 4, 4,  ..., 3, 3, 3])

如果你想要載入更多其他已經準備好的Dataset，請參照[這裡](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html)。當然如果你有自己的數據，pytorch-geometric也有提供自定義的Graph Dataset，方法請詳見[教程](https://pytorch-geometric.readthedocs.io/en/latest/notes/create_dataset.html)。

## Graph Convolution Network (GCN)

